In [7]:
from keras.datasets import mnist
import numpy as np
import sys
import matplotlib.pyplot as plt
np.random.seed(1)

(x_train,y_train),(x_test,y_test) = mnist.load_data()
images ,labels= (x_train[0:1000].reshape(1000,28 * 28)/255, y_train[0:1000])

one_hot_labesls = np.zeros((len(labels),10))
for i,l in enumerate (labels):
    one_hot_labesls[i][l] = 1
labels = one_hot_labesls

test_images = x_test.reshape(len(x_test),28*28)/255
test_labels =  np.zeros((len(y_test),10))
for i,j in enumerate (y_test):
    test_labels[i][j] = 1


In [8]:
tanh =  lambda x:np.tanh(x)
tanh2deriv =  lambda output : (1 - (output ** 2))
def softmax(x):
    temp = np.exp(x)
    return (temp / (np.sum(temp,axis =1, keepdims = True)))

In [9]:
alpha = 2
iterations = 300
pixels_per_image ,num_labels = 784,10
batch_size = 128

input_rows = 28
input_columns = 28

kernel_rows = 3
kernel_col = 3
num_of_kernels = 16

hidden_size = ((input_rows - kernel_rows) * (input_columns - kernel_col)) * num_of_kernels
#print(hidden_size)
kernels = 0.02 * np.random.random((kernel_rows * kernel_col,num_of_kernels)) - 0.01
#print(kernels.shape)
weights_1_2 = 0.2 * np.random.random((hidden_size,num_labels)) - 0.01

def get_image_section(layer,row_from,row_to,col_from,col_to):
    sect = layer[:,row_from : row_to,col_from: col_to]
    return sect.reshape(-1,1,row_to - row_from, col_to - col_from)

In [ ]:
performance = list()
x_xes = list()
for j in range (iterations):
    cnt = 0
    for i  in range (int(len(images)/batch_size)):
        batch_start,batch_end = ((i * batch_size) , ((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 =  layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape
        
        sects = list()
        for row_start in range (layer_0.shape[1] - kernel_rows):
            for col_start in range (layer_0.shape[2] - kernel_col):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_col)
                sects.append(sect)
        expanded_input = np.concatenate(sects,axis = 1)
        es = expanded_input.shape
        flattenned_input = expanded_input.reshape(es[0]*es[1],-1)
        kernel_output = flattenned_input.dot(kernels)
        #print(kernel_output.shape)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        drop_out_mask = np.random.randint(2 ,size = layer_1.shape)
        layer_1  *= drop_out_mask * 2
        layer_2 = softmax (np.dot(layer_1 ,weights_1_2))
        for k in range (batch_size):
            label_set = labels[k+batch_start:k+1+batch_start]
            cnt += int(np.argmax(label_set) == np.argmax(layer_2[k:k+1]))
        
        layer_2_delta = (labels[batch_start:batch_end] - layer_2)/(batch_size * layer_2.shape[0])
        layer_1_delta = (layer_2_delta.dot(weights_1_2.T)) * tanh2deriv(layer_1)
        layer_1_delta *=  drop_out_mask
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        lid_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattenned_input.T.dot(lid_reshape)
        kernels -= alpha * (k_update)
    
    test_correct_cnt = 0
    for i in range (len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        sects = list()
        for row_start in range(layer_0.shape[1]- kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_col):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start + kernel_rows,
                                         col_start,
                                         col_start + kernel_col)
                sects.append(sect)
        expanded_input = np.concatenate(sects,axis = 1)
        es = expanded_input.shape
        flattenned_input = expanded_input.reshape(es[0]*es[1],-1)
        kernel_output = flattenned_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        
    if (j %1 == 0):
        sys.stdout.write("\n"+\
                             "Epoch :"+ str(j)+\
                            " Test Acc :"+ str(test_correct_cnt/float(len(test_images)))+\
                            "Train Acc :"+ str(cnt/float(len(images))))
        performance.append (test_correct_cnt/float(len(test_images)))
        x_xes.append(j)

                


Epoch :0 Test Acc :0.0273Train Acc :0.037
Epoch :1 Test Acc :0.028Train Acc :0.037
Epoch :2 Test Acc :0.0292Train Acc :0.04
Epoch :3 Test Acc :0.0339Train Acc :0.046
Epoch :4 Test Acc :0.0478Train Acc :0.068
Epoch :5 Test Acc :0.076Train Acc :0.083
Epoch :6 Test Acc :0.1316Train Acc :0.096
Epoch :7 Test Acc :0.2137Train Acc :0.127
Epoch :8 Test Acc :0.2941Train Acc :0.148
Epoch :9 Test Acc :0.3563Train Acc :0.181
Epoch :10 Test Acc :0.4023Train Acc :0.209
Epoch :11 Test Acc :0.4358Train Acc :0.238
Epoch :12 Test Acc :0.4473Train Acc :0.286
Epoch :13 Test Acc :0.4389Train Acc :0.274
Epoch :14 Test Acc :0.3951Train Acc :0.257
Epoch :15 Test Acc :0.2222Train Acc :0.243
Epoch :16 Test Acc :0.0613Train Acc :0.112
Epoch :17 Test Acc :0.0266Train Acc :0.035
Epoch :18 Test Acc :0.0127Train Acc :0.026
Epoch :19 Test Acc :0.0133Train Acc :0.022
Epoch :20 Test Acc :0.0185Train Acc :0.038
Epoch :21 Test Acc :0.0363Train Acc :0.038
Epoch :22 Test Acc :0.0928Train Acc :0.067
Epoch :23 Test Acc :0.1

In [ ]:
plt.plot(x_xes,performance,color = 'green')